In [4]:
import os
print(os.listdir())


['.config', 'Chatbot.ipynb', 'intents.json', 'sample_data']


In [5]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
from nltk_utils import tokenize, stem, bag_of_words

In [6]:
ignore_words = ["?", "i", "it", "a", "!", "me", "you", "'s", "does", "of", "do", "my", "are" ]

In [8]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Traning ANN Model


```
# This is formatted as code
```



In [9]:
with open("intents.json", "r") as f:
    intents = json.load(f)

tags = []
all_words = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)

    for pattern in intent["patterns"]:

        words = tokenize(pattern)
        l1 = [stem(w) for w in words if w.lower() not in ignore_words]

        all_words.extend(l1)
        xy.append((l1, tag))

all_words = sorted(set(all_words))
np.random.shuffle(xy)

In [10]:
print(tags)

['unknown', 'greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']


In [11]:
print(all_words)

['appreci', 'avail', 'buy', 'bye', 'can', 'card', 'cash', 'credit', 'deliv', 'deliveri', 'expect', 'funni', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'is', 'item', 'joke', 'know', 'later', 'laugh', 'lot', 'make', 'okay', 'on', 'packag', 'pay', 'payment', 'paypal', 'product', 'realli', 'reciev', 'sell', 'ship', 'should', 'someth', 'thank', 'that', 'the', 'time', 'up', 'use', 'what', 'when', 'will']


In [12]:
print(xy)

[(['thank'], 'thanks'), (['avail'], 'items'), (['deliv'], 'delivery'), (['have', 'cash', 'on', 'deliveri'], 'payments'), (['laugh'], 'funny'), (['sell'], 'items'), (['can', 'use', 'paypal'], 'payments'), (['know', 'joke'], 'funny'), (['thank', 'lot'], 'thanks'), (['how'], 'greeting'), (['hi'], 'greeting'), (['buy'], 'items'), (['later'], 'goodbye'), (['hey'], 'greeting'), (['how', 'should', 'pay'], 'payments'), (['item'], 'items'), (['how', 'can', 'make', 'payment'], 'payments'), (['what', 'up'], 'greeting'), (['thank'], 'thanks'), (['funni', 'joke'], 'funny'), (['joke'], 'funny'), (['what', 'is', 'the', 'expect', 'deliveri', 'time'], 'delivery'), (['realli', 'appreci'], 'thanks'), (['someth', 'funni'], 'funny'), (['goodby'], 'goodbye'), (['ship'], 'delivery'), (['product'], 'items'), (['hello'], 'greeting'), (['can', 'use', 'credit', 'card'], 'payments'), (['that', 'help'], 'thanks'), (['when', 'will', 'reciev', 'packag'], 'delivery'), (['bye'], 'goodbye'), (['okay'], 'thanks')]


In [13]:
df = pd.DataFrame(xy)
df.head()

,0,1
0,[thank],thanks
1,[avail],items
2,[deliv],delivery
3,"[have, cash, on, deliveri]",payments
4,[laugh],funny


In [14]:
X_train = []
y_train = []

for sentence, tag in xy:
    bag = bag_of_words(sentence, all_words)
    X_train.append(bag)

    label = bag_of_words(tag, tags)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [15]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
y_train

array([[0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 

In [17]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=16, input_shape = np.shape(all_words), activation='relu'))
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

ann.add(tf.keras.layers.Dense(units=len(tags) , activation='softmax'))

ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [18]:
ann.fit(X_train, y_train, batch_size=6, epochs = 200)

Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 2.0896 - accuracy: 0.1818
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0760 - accuracy: 0.1818
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0654 - accuracy: 0.2121
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0539 - accuracy: 0.2424
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0437 - accuracy: 0.2727
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0326 - accuracy: 0.3030
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0218 - accuracy: 0.2727
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0111 - accuracy: 0.2727
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0010 - accuracy: 0.2727
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.9913 - accuracy: 0.2424
Epoch 11/200
6/6 [===========

Results

In [19]:
input_1 = "hey"
np.argmax(ann.predict(bag_of_words(input_1, all_words).reshape(1, -1)))

1/1 [==============================] - 0s 87ms/step


1

In [20]:
def respond(sentence):


    words = tokenize(sentence)
    sentence = [stem(w) for w in words if w not in ignore_words]
    bag = bag_of_words(sentence, all_words).reshape(1, -1)

    probs = ann.predict(bag)

    if max(probs[0]) < 0.3 :
        max_idx = 0
    else :
        max_idx = np.argmax(probs)
    response = np.random.choice(intents['intents'][max_idx]['responses'])
    tag = intents['intents'][max_idx]['tag']

    return response, tag

In [22]:
input_2 = "i really appriciate it "

respond(input_2)

1/1 [==============================] - 0s 45ms/step


('My pleasure', 'thanks')